<div>
  <h1>Game Theory</h1>

  <p>Sander Aarts, 2020 (Based on <i>Lab 11: Matrix Games</i> by David Williamson, 2013)</p>
</div>  

<div>
  <p><b>Objectives:</b></p>

  <ul>
    <li>Introduce students to basic concepts in $0$-sum $2$-person games</li>
    <li>Give students experience at playing games and computing optimal mixed minmax strategies</li>
    <li>Help students understand the weaknesses of certain types of strategies</li>
  </ul>

  <p><b>Reading:</b> The data in this lab was found from the following Stats Zone articles: <a href="https://www.thestatszone.com/archive/statistical-insight-into-penalty-shootouts-where-to-place-your-penalty-13886">Article 1</a> and <a href="https://www.thestatszone.com/archive/an-analyses-of-penalty-taking-trends-in-the-champions-league-13654">Article 2</a></p>

  <p><b>Brief description:</b> In this lab, you will consider soccer penalty kicks as simple games. Following an introduction to the game's rules and setting, you will play several rounds against the computer. 
You will then formally solve the game by describing the player's strategies, and characterizing the game's equilibrium strategies and payoffs. Finally, you will explore the effects that different strategies used by the computer have on the viability of your determined mixed strategy.</p>

  <font color='blue'><p><b>Solutions are shown blue.</b></p></font>
  <font color='red'><p><b>Instuctor comments are shown in red.</b></p></font>

  <font color='red'><p>NOTE: As each round has an even number of iterations, odd scores are unattainable. Any student claiming they have one is incorrect.</p></font>
  <font color='red'><p>NOTE: I found this <a href="https://www.researchgate.net/publication/329538967_On_the_One-shot_Two-person_Zero-sum_Game_in_Football_from_a_Penalty_Kicker's_Perspective">paper</a> that I believe has a fairly similar objective to this lab, and ultimately suggests that the kicker should use a mixed strategy where they should be more likely to aim in the direction that the goal keeper is least likely to save the ball, based off of their previous penalty kick experiences (I thought this was interesting, as it seems like fairly similar reasoning as to why we had the goalie use fictitious play).</p></font>
</div>

In [1]:
# Setup -- DON'T FORGET TO RUN THIS CELL.
## Imports:
import asyncio
import pandas as pd
import penaltykickgame as game
from ortools.linear_solver import pywraplp as OR
from bokeh.io import show, output_notebook, save, output_file

## Configure Bokeh for Jupyter Notebook:
output_notebook()

## Set up the demo maker:
demo_maker = game.main_game.MainGame()

## Make each demo:
demo1 = await demo_maker.make_game(
    CPU_strategy="Random", force_fast_forward=True, force_fast_forward_spd=100,
    iterations_to_run=50, stats_fig_1_enabled=False, stats_fig_2_enabled=False,
    stats_fig_3_enabled=False, stats_fig_4_enabled=False
)
demo2 = await demo_maker.make_game(
    CPU_strategy="Random", force_fast_forward=True, force_fast_forward_spd=100,
    iterations_to_run=50
)
demo3 = await demo_maker.make_game(
    CPU_strategy="Mixed_Strategy", force_fast_forward=True, force_fast_forward_spd=100,
    iterations_to_run=50
)
demo4 = await demo_maker.make_game(
    CPU_strategy="Fictitious_Play", force_fast_forward=True, force_fast_forward_spd=50,
    iterations_to_run=50
)
demo5 = await demo_maker.make_game(
    CPU_strategy="Fictitious_Play", force_fast_forward=True, force_fast_forward_spd=50,
    iterations_to_run=100
)
demo6 = await demo_maker.make_game(
    CPU_strategy="Fictitious_Play", force_fast_forward=True, force_fast_forward_spd=50,
    iterations_to_run=200
)
demo7 = await demo_maker.make_game(
    CPU_strategy="Fictitious_Play", force_fast_forward=True, force_fast_forward_spd=0,
    iterations_to_run=500
)
demo8 = await demo_maker.make_game(
    CPU_strategy="Goalie_Cheats", force_fast_forward=True, force_fast_forward_spd=50,
    iterations_to_run=50
)

Loading BokehJS ...

<div>
  <h2>Part 1: The Penalty Kick Game</h2>

  <p>In this part of the lab, you will familiarize yourself with the penalty kick game.</p>
</div>

<div>
  <h3>The Setting</h3>

  <p>In soccer (or football), a team may be awarded a penalty kick for being fouled within their opposition's penalty area. The awarded team selects a player (the <i>kicker</i>) to engage in a $1$-shot, $2$-person, $0$-sum game against the opposing goal keeper (their <i>keeper</i> for short). The kicker aims to kick the ball into the goal, while the keeper tries to block it. Both player's actions are predetermined, as the ball typically takes about $0.3$ seconds to reach the goal once kicked.</p>
</div>

<br>

<div style="align: left; text-align:center;">
  <img src="images/carli_lloyd_goal.gif" alt="Carli Lloyd (US) scoring a penalty kick against Germany, 2015" title="Carli Lloyd (US) scoring a penalty kick against Germany, 2015." width="360px"/>

  <div class="caption">Figure $1$: A penalty kick. The kicker kicks to the right; the keeper dives to the left. It's a goal!</div>
</div>

<a id="Rules"></a>
<div>
  <h3>The Rules</h3>

  <p>You will play a simple version of the penalty kick game. The rules are as follows:</p>

  <ul>
    <li>You will play multiple iterations as the kickers to create a total score that you wish to maximize.</li>
    <li>For each iteration, you will select a <b>left or right -footed kicker</b>. The keeper will be informed of your choice.</li>
    <li>You and the keeper will then both commit to playing (kicker kicks, keeper dives) <b>left, middle, or right</b></li> 
    <li>The keeper's action, the kicker's footedness, and their kick direction determine the goal scoring probability according to Table 1.</li>
    <li>A shot result is drawn according to this probability. A goal generates $1$ point; a save $-1$.</li>
  </ul>
</div>

<br>

<div style="align: left; text-align:center;">
  <table style="width:60%">
    <tr>
      <th>Scoring Probability</th>
      <th>LL</th>
      <th>LM</th>
      <th>LR</th>
      <th>ML</th>
      <th>MM</th>
      <th>MR</th>
      <th>RL</th>
      <th>RM</th>
      <th>RR</th>
    </tr>
    <tr>
      <td><b>Right-footed kicker</b></td>
      <td>$0.55$</td>
      <td>$0.65$</td>
      <td>$0.93$</td>
      <td>$0.74$</td>
      <td>$0.60$</td>
      <td>$0.72$</td>
      <td>$0.95$</td>
      <td>$0.73$</td>
      <td>$0.70$</td>
    </tr>
    <tr>
      <td><b>Left-footed kicker</b></td>
      <td>$0.67$</td>
      <td>$0.70$</td>
      <td>$0.96$</td>
      <td>$0.74$</td>
      <td>$0.60$</td>
      <td>$0.72$</td>
      <td>$0.87$</td>
      <td>$0.65$</td>
      <td>$0.61$</td>
    </tr>
  </table>
  <div class="caption">Table $1$: The Column headers indicate kicker kick, keeper dive directions. </div>
</div> 

<div><h3>Your first attempt</h3></div>

<div>
  <p><b>Q1:</b> Play $30$ rounds against the computer <a href="http://engri1101.orie.cornell.edu/penalty_kick_source/penalty_kick_game.html">HERE</a>. The keeper will use <i>fictitious play</i> (Explained in <a href="#Part 4: Weaknesses of Mixed Strategies">Part 4</a>) to attempt to counter your strategy.</p>

  <p>How high did your score?</p>

  <p><b>A:</b><font color='blue'> Answers will vary.</font></p>
</div>

<div>
  <br>
  <p><big><strong>The rest of the lab restricts you to using a mixed strategy for the kicker's actions.</strong></big></p>
</div>

<div>
  <br>
  <h2>Part 2: Solving the Penalty Kick Game</h2>

  <p>In this part of the lab, we will analyze the mixed strategy penalty kick game.</p>
</div>

<div>
  <h3>Identifying the Pure Strategies</h3>

  <p>The first step of formulating a $2$-persion $0$-sum game is to figure out <b>each player's pure strategies</b>.</p>

  <p><i>Remember: A pure strategy is a complete set of playable instructions formulating a feasible strategy for a player to take in the game.</i></p>
</div>

<div>
  <p><b>Q2:</b> What are the kicker's pure strategies?</p>

  <p><b>A:</b></p>
  <font color='blue'><ul>
    <li>Pick a <b>left-footed</b> kicker instructed to <b>kick to the left</b></li>
    <li>Pick a <b>left-footed</b> kicker instructed to <b>kick to the middle</b></li>
    <li>Pick a <b>left-footed</b> kicker instructed to <b>kick to the right</b></li>
    <li>Pick a <b>right-footed</b> kicker instructed to <b>kick to the left</b></li>
    <li>Pick a <b>right-footed</b> kicker instructed to <b>kick to the middle</b></li>
    <li>Pick a <b>right-footed</b> kicker instructed to <b>kick to the right</b></li>
  </ul></font>
</div>

<div>
  <p>As the keeper's action can be chosen after the kicker's dominant foot is announced, their pure strategies are slightly more complicated. The following questions will walk you through determining the keeper's pure strategies.</p>
</div>

<div>
  <p><b>Q3:</b> Which choices does the keeper require a specified response for?</p>

  <p><b>A:</b><font color='blue'> The keeper requires specified responses for <b>left</b> and <b>right -footed kickers</b>.</font></p>
</div>

<div>
  <p>Following the notation from the handout, a pure strategy from the keeper takes the form $([L, i], [R, j])$ read as:</p>

  <p><i>If the kicker is left-footed, do $i$. If the kicker is right-footed, do $j$.</i></p>
</div>

<div>
  <p><b>Q4:</b> What values can $i$ and $j$ take? How many pure strategies does the keeper have?</p>

  <p><b>A:</b><font color='blue'> Both $i$ and $j$ can each take the values ${L, M, R}$. Hence, the keeper has $9$ pure strategies:</font></p>
  <font color='blue'><ol type="1">
    <li>$([L, L], [R, L])$</li>
    <li>$([L, L], [R, M])$</li>
    <li>$([L, L], [R, R])$</li>
    <li>$([L, M], [R, L])$</li>
    <li>$([L, M], [R, M])$</li>
    <li>$([L, M], [R, R])$</li>
    <li>$([L, R], [R, L])$</li>
    <li>$([L, R], [R, M])$</li>
    <li>$([L, R], [R, R])$</li>
  </ol></font>
</div>

<div>
  <h3>Attempting a Mixed Strategy</h3>

  <p>The following questions walk you through trying several mixed strategies for the penalty kick game.</p>

  <p><i>Remember: A mixed strategy is a strategy where pure strategies are used at varying pre-determined probabilities.</i></p>
</div>

<div>
  <p><b>Q5:</b> Give an example of a valid mixed strategy for the game.</p>

  <p><b>A:</b><font color='blue'> Answers will vary. Example answer:</font></p>
  <font color='blue'><ul>
    <li>$33$% chance <b>left-footed</b> kicker <b>kicking to the left</b>
    <li>$17$% chance <b>right-footed</b> kicker <b>kicking to the left</b>
    <li>$50$% chance <b>right-footed</b> kicker <b>kicking to the right</b>
  </ul></font>
</div>

<div>
  <p><b>Q6:</b> Try running the mixed strategy penalty kick game from the cell below. <b>For this version, you will input each pure strategy's selection chance as a decimal between $0$ and $1$.</b> The keeper will pick their actions at random. The game will also fast forward between iterations.</p>

  <p>How high did you score?</p>

  <p><b>A:</b><font color='blue'> Answers will vary.</font></p>
</div>

In [2]:
# Shows the game so that you can play it
show(demo1)

<div style="align: left; text-align:center;">
  <table style="width:60%">
    <tr>
      <th>Scoring Probability</th>
      <th>LL</th>
      <th>LM</th>
      <th>LR</th>
      <th>ML</th>
      <th>MM</th>
      <th>MR</th>
      <th>RL</th>
      <th>RM</th>
      <th>RR</th>
    </tr>
    <tr>
      <td><b>Right-footed kicker</b></td>
      <td>$0.55$</td>
      <td>$0.65$</td>
      <td>$0.93$</td>
      <td>$0.74$</td>
      <td>$0.60$</td>
      <td>$0.72$</td>
      <td>$0.95$</td>
      <td>$0.73$</td>
      <td>$0.70$</td>
    </tr>
    <tr>
      <td><b>Left-footed kicker</b></td>
      <td>$0.67$</td>
      <td>$0.70$</td>
      <td>$0.96$</td>
      <td>$0.74$</td>
      <td>$0.60$</td>
      <td>$0.72$</td>
      <td>$0.87$</td>
      <td>$0.65$</td>
      <td>$0.61$</td>
    </tr>
  </table>
  <div class="caption">Table $1$: The Column headers indicate kicker kick, keeper dive directions.</div>
</div>

<br>

<div>
  <p><b>Q7:</b> As stated in <a href="#Rules">the rules</a>, different kicker and keeper choice combinations have varying goal scoring chances. Use Table $1$ and the demo below to improve your mixed strategy (Don't spend too long doing this).</p>

  <p>The demo below also contains the 3 following figures (Fig $3$ is irrelevant to the section and is hidden):</p>
  <ul>
    <li>Fig $1$: Bars representing the actions taken by the kicker with sections displaying information on their outcomes.</li>
    <li>Fig $2$: Score over iterations run (Red dots), with black dots showing expected score (if average luck) calculated from previous score chances (black dot values aren't attainable, but indicate strategy efficacy without relying on score roll luck).</li>
    <li>Fig $4$: Iteration score chances (Yellow dots), with game average score chance (black line) (Note: average score chance should be represented dots, but the line was used for visualization).</li>
  </ul>

  <p>How high did you score? What was your best mixed strategy?</p>

  <p><b>A:</b><font color='blue'> Answers will vary.</font></p>
</div>

In [4]:
# Shows the game so that you can play it. CAN BE RE-RUN TO REDO THE DEMO.
show(demo2)

<div>
  <h3>Analyzing The Payout Matrices</h3>

  <p>Now, we will break the game down into the <i>left</i> and <i>right -footed kicker subgames</i> to simplify the problem.</p>
</div>

<div style="align: left; text-align:center;">
  <table style="width:60%">
    <tr>
      <th>Scoring Probability</th>
      <th>LL</th>
      <th>LM</th>
      <th>LR</th>
      <th>ML</th>
      <th>MM</th>
      <th>MR</th>
      <th>RL</th>
      <th>RM</th>
      <th>RR</th>
    </tr>
    <tr>
      <td><b>Right-footed kicker</b></td>
      <td>$0.55$</td>
      <td>$0.65$</td>
      <td>$0.93$</td>
      <td>$0.74$</td>
      <td>$0.60$</td>
      <td>$0.72$</td>
      <td>$0.95$</td>
      <td>$0.73$</td>
      <td>$0.70$</td>
    </tr>
    <tr>
      <td><b>Left-footed kicker</b></td>
      <td>$0.67$</td>
      <td>$0.70$</td>
      <td>$0.96$</td>
      <td>$0.74$</td>
      <td>$0.60$</td>
      <td>$0.72$</td>
      <td>$0.87$</td>
      <td>$0.65$</td>
      <td>$0.61$</td>
    </tr>
  </table>
  <div class="caption">Table $1$: The Column headers indicate kicker kick, keeper dive directions.</div>
</div>

<br>

<div>
  <p><b>Q8:</b> Use Table 1 to write down the $3$x$3$ payoff matrices for <b>both</b> games.</p>

  <p><b>A:</b><font color='blue'> Answer revealed below.</font></p>
</div>

<div>
  <p><b>Q9:</b> In each matrix, do any pure strategies dominate another pure strategy?</p>

  <p><b>A:</b><font color='blue'> No.</font></p>
</div>

<div>
  <p>Run the following cell to see the correct payoff matrices for both of the sub-games. Here, the kicker is the row player and the keeper is the column player. <b>If they do not match your answer, consider why. Use these payoff matrices for the following computations.</b></p>
</div>

In [5]:
left = pd.read_csv("data/left_subgame.csv", index_col=0)
right = pd.read_csv("data/right_subgame.csv", index_col=0)
display(left, right)

,,,
,,,
Left-footed kicker,L,M,R
L,0.67,0.7,0.96
M,0.74,0.6,0.72
R,0.87,0.65,0.61


,,,
,,,
Right-footed kicker,L,M,R
L,0.55,0.65,0.93
M,0.74,0.6,0.72
R,0.95,0.73,0.7


<div>
  <br>
  <p><big><strong>In lecture and the handout, you learned about dominance through pure strategies. Similarly, Mixed strategies can also achieve dominance.</strong></big></p>

  <p><b>First, consider the right-footed kicker sub-game.</b><p>
</div>

<div>
  <p><b>Q10:</b> What are the expected payoffs for the kicker's $50/50$ mixed strategy between kicking to the left or right? You should give the strategy's expected payoffs for each of the possible keeper actions (diving to the left ($L$), middle ($M$), or right ($R$)).</p>

  <p><b>A:</b><font color='blue'> The expected payoffs are:<br>
$(L) : \frac{1}{2}(0.55) + \frac{1}{2}(0.95) = 0.75$<br>
$(M) : \frac{1}{2}(0.65) + \frac{1}{2}(0.73) = 0.69$<br>
$(R) : \frac{1}{2}(0.93) + \frac{1}{2}(0.70) = 0.815$</font></p>
</div>

<div>
  <font color='red'><p>Possibly combine with <b>Q10</b>?</p></font>

  <p><b>Q11:</b> Does the kicker's $50/50$ mixed strategy from <b>Q10</b> dominate any of their pure strategies? If so, which?</p>

  <p><b>A:</b><font color='blue'> It dominates the pure strategy of kicking to the middle, as its ($L$, $M$, $R$) payoff values are higher.</font></p>
</div>

<a id="right sub -r_M"></a>
<div>
  <p><b>You should have found that the pure strategy of kicking to the middle is dominated. As it will therefore never be played, it can be removed from the payoff matrix (See cell below).</b></p>
</div>

In [6]:
# Remove kicker kicks to the middle pure strategy
right.drop('M')

,,,
,,,
Right-footed kicker,L,M,R
L,0.55,0.65,0.93
R,0.95,0.73,0.7


<div>
  <p><b>Q12:</b> Does the keeper's $50/50$ mixed strategy between diving left or right dominate their pure strategy of staying in the middle? Why or why not?</p>

  <p><b>A:</b><font color='blue'> <b>No.</b> The mixed strategy's expected payoffs for the kicker's different actions are:<br>
$(L) : \frac{1}{2}(0.55) + \frac{1}{2}(0.93) = 0.74$<br>
$(R) : \frac{1}{2}(0.95) + \frac{1}{2}(0.70) = 0.825$</font></p>

  <font color='blue'><p>As the mixed strategy's expected payoff is higher for at least $1$ kicker action, and the keeper's aim is to minimize the kicker's goal scoring chance, <b>it does not dominate the pure strategy of staying in the middle.</b></p></font>
</div>

<div><p><b>Now consider the left-footed kicker sub-game.</b></p></div>

<div>
  <p><b>Q13:</b> Simplify the payoff matrix to a $3$x$2$ by eliminating any dominated pure strategies. (Hint: It simplifies in a similar fashion to the right-footed kicker payoff matrix.)</p>

  <p><b>A:</b><font color='blue'> Again, the kicker's $50/50$ mixed strategy between kicking to the left or right dominates their pure strategy of staying in the middle.</font></p>
</div>

In [7]:
# TODO: DROP A DOMINATED ROW FROM THE LEFT PAYOFF MATRIX
# left.drop('L'), left.drop('M'), or left.drop('R')

### BEGIN SOLUTION
left.drop('M')
### END SOLUTION

,,,
,,,
Left-footed kicker,L,M,R
L,0.67,0.7,0.96
R,0.87,0.65,0.61


<div>
  <h3>Solving for the Optimal Mixed Strategy</h3>

  <p><b>Let's return to the right-footed kicker sub-game.</b></p>
<div>

In [8]:
# Right-foot kicker payoff matrix for reference
right.drop('M')

,,,
,,,
Right-footed kicker,L,M,R
L,0.55,0.65,0.93
R,0.95,0.73,0.7


<div>
  <p><b>Q14:</b> If the kicker kicks to the left and right with probabilities $p_L$ and $p_R$, what are their expected payoffs associated with each of the keeper's pure strategies ($L$, $M$, $R$)?</p>

  <p><b>A:</b><font color='blue'> The expected payoffs are:<br>
$(L) : 0.55p_L + 0.95p_R$<br>
$(M) : 0.65p_L + 0.73p_R$<br>
$(R) : 0.93p_L + 0.70p_R$</font></p>
</div>

<div>
  <p><b>Q15:</b> Simplify your expressions from <b>Q14</b> to depend solely on the variable $p_L$. (Hint: What is the sum of $p_L$ and $p_R$?)</p>

  <p><b>A:</b><font color='blue'> $p_L + p_R = 1$</font></p>

  <font color='blue'><p>$p_R = (1 - p_L)$</p></font>

  <font color='blue'><p>$(L) : 0.55p_L + 0.95p_R = 0.55p_L + 0.95(1 - p_L) = 0.55p_L + 0.95 - 0.95p_L = 0.95 - 0.40p_L$<br>
  $(M) : 0.65p_L + 0.73p_R = 0.65p_L + 0.73(1 - p_L) = 0.65p_L + 0.73 - 0.73p_L = 0.73 - 0.08p_L$<br>
  $(R) : 0.93p_L + 0.70p_R = 0.93p_L + 0.70(1 - p_L) = 0.93p_L + 0.70 - 0.70p_L = 0.70 + 0.23p_L$</p></font>
</div>

<div>
  <p><b>Q16: Using the graphical method</b>, find the values of $p_L$ and $p_R$ that maximize the minimum goal scoring probability.</p>

  <p>What is the kicker's corresponding optimal mixed strategy? What is its minimum goal scoring probability? (<a href="https://www.desmos.com/calculator">Desmos</a> is a helpful tool!)</p>

  <p><b>A:</b> <font color='blue'><b>Solution:<br>
$p_L = 0.0968$, $p_R = 0.9032$<br>
With a scoring probability of $0.7223$.</b></font></p>

  <font color='blue'><p>This can be obtained by finding the highest point (located at $(0.0968, 0.7223)$) that satisfies the following conditions in desmos:<br>
$0 \leq y \leq 0.95 - 0.4x${$0 \leq x \leq 1$}<br>
$0 \leq y \leq 0.73 - 0.08x${$0 \leq x \leq 1$}<br>
$0 \leq y \leq 0.70 + 0.23x${$0 \leq x \leq 1$}</p></font>

  <font color='blue'><p>Since $x$ was used to represent $p_L$ and y was used to represent scoring probability:<br>
$p_L = 0.0968$, $p_R = 1-0.0968 = 0.9032$ and scoring probability $= 0.7223$.</p></font>

  <font color='blue'><p>Please note, other methods of plotting this are also most definitely valid.</p></font>
</div>

<div>
  <font color='red'><p>This actually isn't necessarily correct. Lines for enforcing $0 \leq x \leq 1$ and $0 \leq y$ could also be included ($y \leq 1$ isn't really needed as the other constraints already enforce it).</p></font>

  <p><b>Q17:</b> In <b>Q16</b> you should have found the kicker's optimal mixed strategy by graphing $3$ lines. What did they correspond to?

  <p><b>A:</b><font color='blue'> Each line corresponded to a different pure strategy from the keeper.</font></p>
</div>

<div>
  <p>Alternatively, the kicker's optimal mixed strategy can be found by solving to maximize the minimmum goal scoring probability through linear programming. This can be done by maximizing the value of a new variable $z$ constrained to be $\leq$ all $3$ expected payoffs, as shown by the following linear program:</p>

  <p>$$\begin{align*}
\max \quad & z\\
\text{s.t.} \quad & z \leq 0.55p_L + 0.95p_R\\
\quad & z \leq 0.65p_L + 0.73p_R\\
\quad & z \leq 0.93p_L + 0.70p_R\\
\quad & p_L + p_R = 1 \\
\quad & 0\leq z, p_L, p_R \leq 1 \\
\end{align*}$$</p>

  <p><b>Where $p_L$ and $p_R$ are the kicker's chances of kicking to the left and right.</b></p>

  <p>Run the cells below to solve the linear program. Verify that your solution to <b>Q16</b> matches its solution.</p>
<div>

In [9]:
def right_subgame_kicker():
    """A linear program for solving the right penalty kick subgame for the kicker."""
    # define the model
    m = OR.Solver("right_subgame-kicker", OR.Solver.CLP_LINEAR_PROGRAMMING);

    # decision variables
    L = m.NumVar(0, 1, "L"); # Chance of the kicker kicking left. (p_L)
    R = m.NumVar(0, 1, "R"); # Chance of the kicker kicking right. (p_R)
    z = m.NumVar(0, 1, "z");

    # objective function.
    m.Maximize(z); # Maximize minimum chance of scoring.

    # constraints.
    # z <= Chance of scoring if goalie dives to the left.
    m.Add(z <= 0.55*L + 0.95*R);
    
    # z <= Chance of scoring if goalie stays in the middle.
    m.Add(z <= 0.65*L + 0.73*R);
    
    # z <= Chance of scoring if goalie dives to the right.
    m.Add(z <= 0.93*L + 0.70*R);
    
    # Chance of Kicker kicking left and kicker kicking right add up to 1.
    m.Add(L + R == 1);
    
    return m, m.variables()

In [10]:
def solve(m):
    m.Solve()
    print("Solution:")
    print("Objective value =", m.Objective().Value())
    for var in m.variables():
        print(var.name(), ":",  var.solution_value())

In [11]:
m,x = right_subgame_kicker()
solve(m)

Solution:
Objective value = 0.722258064516129
L : 0.09677419354838684
R : 0.9032258064516132
z : 0.722258064516129


<div>
  <p><b>Consider the dual values of the previous LP.</b> Recall the economic interpretation of dual values taught to you in lecture. The dual value of a constraint can be interpretted as the marginal cost of changing the requirement (i.e. the RHS) by $1$. This interpretation has an interesting corollary: <b>Slight changes to a constraint satisfied with strict inequality ($>$ or $<$) in an optimal solution will not affect the optimal solution, and thus, the optimal objective value.</b></p>
</div>

<div>
  <p><b>Q18:</b> Take this one step further. If slight changes to a constraint have no effect on the optimal objective value, what does that imply about the <i>marginal cost</i> of changing this requirement? What does that imply about the dual value of this constraint?</p>

  <p><b>A:</b><font color='blue'> The marginal cost of changing this requirement is $0$ and the dual value of the constraint is $0$.</font></p>
</div>

<div>
  <font color='red'><p>Where did we see this? I know its true, but will the students recognize this by this point?</p></font>

  <p><b>In Q17, you should have noticed that each constraint corresponds to a pure strategy from the keeper.</b> Furthermore, we have seen that the LPs for finding the optimal mixed strategies for the kicker and keeper are each other's duals. Hence, the optimal dual values of the LP's constraints correspond to an optimal mixed strategy for the keeper.</p>
</div>

<div>
  <p><b>Q19: Consider the LP we used to find the kicker's optimal mixed strategy.</b> Which constraint holds with strict inequality in the optimal solution? What is the keeper's corresponding pure strategy? How often does <b>Q18</b> imply the keeper plays this strategy in the dual optimal solution?</p>

  <p><b>A:</b><font color='blue'> $z \leq 0.55p_L + 0.95p_R$, corresponding to the keeper's pure strategy of diving to the left, holds with strict inequality. <b>Q18</b> implies that the keeper therefore never dives left in the corresponding dual optimal solution.</font></p>
</div>

<div>
  <p>We can now remove the ($L$) column from the payoff matrix, as we know that the keeper has an optimal mixed strategy in which they never dive to the left. This allows us to solve for the keeper's optimal mixed strategy using the graphical method.</p>
</div>

In [12]:
right.drop('M').iloc[:,1:]

,,
,,
Right-footed kicker,M,R
L,0.65,0.93
R,0.73,0.7


<div>
  <p><b>Q20:</b> If the keeper dives to the middle and right with probabilities $q_M$ and $q_R$, find the values of $q_M$ and $q_R$ that minimize the maximum goal scoring probability <b>using the graphical method.</b></p>

  <p>What is the keeper's corresponding optimal mixed strategy? What is its maximum goal scoring probability? (<a href="https://www.desmos.com/calculator">Desmos</a> is a helpful tool!)</p>

  <p><b>A:</b><font color='blue'> First simplify the equations for the payoffs associated with each kicker action:<br></font></p>

  <font color='blue'><p>$q_R = (1 - q_M)$<br>
<br>
$(L) :  0.65q_M + 0.93q_R = 0.65q_M + 0.93(1 - q_M) = 0.93 - 0.28q_M$<br>
$(R) :  0.73q_M + 0.70q_R = 0.73q_M + 0.70(1 - q_M) = 0.70 + 0.03q_M$</p></font>

  <font color='blue'><p>Find the lowest point satisfying the following conditions(located at $(0.7419, 0.7223)$):<br>
$0.93 - 0.28x \leq y \leq 1${$0 \leq x \leq 1$}<br>
$0.70 + 0.03x \leq y \leq 1${$0 \leq x \leq 1$}</p></font>

  <font color='blue'><p>Since x was used to represent $q_M$:<br>
$q_M = 0.7419$, $q_R = 1 - 0.7419 = 0.2581$<br>
The maximum scoring probability is $0.7223$.</p></font>

  <font color='blue'><p>Please note that there are many other valid ways of obtaining this by plotting.</p></font>
</div>

<div>
  <p><b>Q21:</b> How did the goal scoring probabilities from <b>Q16</b> and <b>Q20</b> relate? Did you expect this?</p>

  <p><b>A:</b><font color='blue'> They are equal, as expected by the minimax theorem.</font></p>
</div>

<div>
  <p><b>For the left-footed kicker sub-game</b>, we will now find a mixed-strategy equilibrium through linear programming. As a brief reminder, the following trick can be helpful in formulating the problem's LP:</p>

  <p><i>When maximizing the minimum of $2$ variables: $x$, and $y$, we can notationally say $\max_{x,y} \min\{x,y\}$. This can be achieved in an LP through the inclusion of a new variable $z$ in the way shown by the LP below:</i></p>

  <p>$$\begin{align*}
\max \quad & z\\
\text{s.t.} \quad & z\leq x\\
\quad & z \leq y\\
\quad  ...
\end{align*}$$</p>
</div>

In [13]:
# Left-foot kicker payoff matrix for reference
left.drop('M')

,,,
,,,
Left-footed kicker,L,M,R
L,0.67,0.7,0.96
R,0.87,0.65,0.61


<div>
  <p><b>Q22:</b> Assume the kicker kicks to the left and right with probabilities $p_L$ and $p_R$. What is the expected payoff associated with each of the keeper's pure strategies ($L$, $M$, $R$)? (<b>Remember we are thinking about the left-footed kicker now!</b>)</p>

  <p><b>A:</b><font color='blue'> The expected payoffs are:<br>
$(L) : 0.67p_L + 0.87p_R$<br>
$(M) : 0.70p_L + 0.65p_R$<br>
$(R) : 0.96p_L + 0.61p_R$</font></p>
</div>

<div>
  <p><b>Q23:</b> Write a linear program to maximize the kicker's minimum expected goal scoring probability.</p>

  <p><b>A:</b></p>
  <font color='blue'><p>$$\begin{align*}
\max \quad & z\\
\text{s.t.} \quad & z \leq 0.67p_L + 0.87p_R \\
\quad & z \leq 0.70p_L + 0.65p_R\\
\quad & z \leq 0.96p_L + 0.61p_R\\
\quad & p_L + p_R = 1 \\
\quad & 0 \leq z, p_L, p_R \leq 1 \\
\end{align*}$$</p></font>
</div>

<div>
  <p><b>Q24:</b> Implement your solution to <b>Q23</b> in the OR-Tools model below, then run the following cell to solve it.</p>
</div>

In [14]:
def left_subgame_kicker():
    """A linear program for solving the left penalty kick subgame for the kicker."""
    # define the model
    m = OR.Solver("left_subgame-kicker", OR.Solver.CLP_LINEAR_PROGRAMMING);

    # decision variables
    L = m.NumVar(0, 1, "L"); # Chance of kicker kicking left (p_L)
    R = m.NumVar(0, 1, "R"); # Chance of kicker kicking right (p_R) 
    z = m.NumVar(0, 1, "z");

    # objective function
    # TODO: Define the objective function.
    # m.Minimize() or m.Maximize()
    
    ### BEGIN SOLUTION
    m.Maximize(z);
    ### END SOLUTION

    # constraints
    # TODO: Define the constraints.
    # m.Add()
    
    ### BEGIN SOLUTION
    m.Add(z <= 0.67*L + 0.87*R);
    m.Add(z <= 0.70*L + 0.65*R);
    m.Add(z <= 0.96*L + 0.61*R);
    m.Add(L + R == 1);
    ### END SOLUTION
    
    return m, m.variables()

In [15]:
m,x = left_subgame_kicker()
solve(m)

Solution:
Objective value = 0.694
L : 0.8800000000000006
R : 0.11999999999999944
z : 0.694


<div>
  <p><b>Q25:</b> What is an optimal mixed-strategy <b>for the left-footed kicker</b>? What is their minimum expected goal scoring probability?</p>

  <p><b>A:</b><font color='blue'> $p_L = 0.88$ and $p_R = 0.12$. Their minimum expected goal scoring probability is 0.694.</font></p>
</div>

<div>
  <p><b>Q26:</b> Assume the keeper dives to the left, middle, and right, with probabilities $q_L$, $q_M$, and $q_R$. Finish the model below to minimize the kicker's maximum goal scoring probability and obtain the keeper's optimal mixed strategy (Hint: it is the dual of your previous one!).</p>
</div>

In [16]:
def left_subgame_keeper(): 
    """A linear program for solving the left penalty kick subgame for the keeper."""
    # define the model
    m = OR.Solver("left_subgame-keeper", OR.Solver.CLP_LINEAR_PROGRAMMING);

    # decision variables.
    L = m.NumVar(0, 1, "L"); # Chance of keeper diving to the left (q_L)
    M = m.NumVar(0, 1, "M"); # Chance of keeper staying in the middle (q_M)
    R = m.NumVar(0, 1, "R"); # Chance of keeper diving to the right (q_R)
    z = m.NumVar(0, 1, "z");

    # objective function
    # TODO: Define the objective function.
    # m.Minimize() or m.Maximize()

    ### BEGIN SOLUTION
    m.Minimize(z);
    ### END SOLUTION

    # constraints
    # TODO: Define the constraints.
    # m.Add()

    ### BEGIN SOLUTION
    m.Add(z >= 0.67*L + 0.70*M + 0.96*R);
    m.Add(z >= 0.87*L + 0.65*M + 0.61*R);
    m.Add(L + M + R == 1);
    ### END SOLUTION
    
    return m, m.variables()

In [17]:
m,x = left_subgame_keeper()
solve(m)

Solution:
Objective value = 0.694
L : 0.1999999999999998
M : 0.8000000000000002
R : 0.0
z : 0.694


<div>
  <p><b>Q27:</b> What is an optimal mixed-strategy for the keeper <b>against a left-footed kicker</b>? What is their maximum expected probability of being scored on?</p>

  <p><b>A:</b><font color='blue'> $q_L = 0.2$, $q_M = 0.8$, $q_R = 0$ and the maximum expected probability of being scored on is $0.694$.</font></p>
</div>

<div>
  <p><b>Q28:</b> If both types of kickers use their optimal mixed strategies, is one more likely to score?</p>

  <p><b>A:</b><font color='blue'> Yes, the <b>right-footed kicker</b> scores with probability $0.7223 > 0.694$.</font></p>
</div>

<div>
  <p><b>Q29:</b> Use your above findings to compile a <b>complete set of instructions</b> for the players in equilibrium. (Hint: even if a type of kicker is never chosen, you should specify what the goalkeeper should do if they are chosen).</p>

  <p><b>A:</b></p>
  <font color='blue'><ul>
    <li>The kicker is always <b>right-footed</b> and kicks to the <b>left</b> and <b>right</b> with probabilities: $p_L = 0.0968$ and $p_R = 0.9032$</li>
    <li>If the kicker is <b>right-footed</b>, the goalkeeper <b>stays in the middle</b> and <b>dives to the right</b> with probabilities: $q_M = 0.7419$ and $q_R = 0.2581$</li>
    <li>If the kicker is <b>left-footed</b>, the goalkeeper <b>dives to the left</b> and <b>stays in the middle</b> with probabilities: $q_L = 0.2$ and $q_M = 0.8$</li>
  </ul></font>
</div>

<div>
  <font color='red'><p>I want to delete this question.</p></font>

  <p><b>Q30:</b> Does the answer to <b>Q28</b> make intuitive sense?</p>

  <p><b>A:</b><font color='blue'> Yes, the kicker shoots in the dominant foot direction. Unlike real penalty kicks, the keeper often does not dive either direction.</font></p>
</div>

<div>
  <p><b>Q31:</b> In the cell below, use the optimal mixed strategy to play the game. Analyze your results.

  <p><b>A:</b><font color='blue'> Answers will vary.</font></p>
<div>

In [ ]:
# Shows the game so that you can play it
show(demo2)

<div>
  <p><b>Q32:</b> In the cell below, use the optimal mixed strategy to play the game again. This time, the keeper will also use its optimal mixed strategy. How did your results compare to <b>Q31</b>?</p>

  <p><b>A:</b><font color='blue'> Answers will vary. Most answers should say that they did worse (As the keeper should have done a better job of blocking goals).</font></p>
</div>

In [ ]:
# Shows the game so that you can play it
show(demo3)

<div>
  <br>
  <h2>Part 3: Full Model for The Penalty Kick Game Mixed Strategy</h2>

  <p>In <b>Part 2</b>, we divided the problem into subgames before using dominance to simplify the payoff matrices. However, we could have formulated the following LP to solve the model:</p>
</div>

In [ ]:
def penalty_kick_game(integer = False):
    """A linear program for solving a penalty kick game."""
    # define the model
    m = OR.Solver("penalty_kick_game", OR.Solver.CLP_LINEAR_PROGRAMMING);

    LL = m.NumVar(0, 1, "LL"); # Chance of left footed kicker shooting left
    LM = m.NumVar(0, 1, "LM"); # Chance of left footed kicker shooting middle
    LR = m.NumVar(0, 1, "LR"); # Chance of left footed kicker shooting right
    RL = m.NumVar(0, 1, "RL"); # Chance of right footed kicker shooting left
    RM = m.NumVar(0, 1, "RM"); # Chance of right footed kicker shooting middle
    RR = m.NumVar(0, 1, "RR"); # Chance of right footed kicker shooting right

    z = m.NumVar(0, 1, "z"); # Maximize minimum chance of scoring

    # objective function
    m.Maximize(z);

    # constraints
    # z <= Chance of scoring if keeper follows ([L,L],[R,L]) strategy
    m.Add(z <= 0.67*LL + 0.74*LM + 0.87*LR + 0.55*RL + 0.74*RM + 0.95*RR, name="([L,L],[R,L])");
    
    # z <= Chance of scoring if keeper follows ([L,M],[R,L]) strategy
    m.Add(z <= 0.70*LL + 0.60*LM + 0.65*LR + 0.55*RL + 0.74*RM + 0.95*RR, name="([L,M],[R,L])");
    
    # z <= Chance of scoring if keeper follows ([L,R],[R,L]) strategy
    m.Add(z <= 0.96*LL + 0.72*LM + 0.61*LR + 0.55*RL + 0.74*RM + 0.95*RR, name="([L,R],[R,L])");
    
    # z <= Chance of scoring if keeper follows ([L,L],[R,M]) strategy
    m.Add(z <= 0.67*LL + 0.74*LM + 0.87*LR + 0.65*RL + 0.60*RM + 0.73*RR, name="([L,L],[R,M])");    
    
    # z <= Chance of scoring if keeper follows ([L,M],[R,M]) strategy
    m.Add(z <= 0.70*LL + 0.60*LM + 0.65*LR + 0.65*RL + 0.60*RM + 0.73*RR, name="([L,M],[R,M])");
    
    # z <= Chance of scoring if keeper follows ([L,R],[R,M]) strategy
    m.Add(z <= 0.96*LL + 0.72*LM + 0.61*LR + 0.65*RL + 0.60*RM + 0.73*RR, name="([L,R],[R,M])");
    
    # z <= Chance of scoring if keeper follows ([L,L],[R,R]) strategy
    m.Add(z <= 0.67*LL + 0.74*LM + 0.87*LR + 0.93*RL + 0.72*RM + 0.70*RR, name="([L,L],[R,R])");
    
    # z <= Chance of scoring if keeper follows ([L,M],[R,R]) strategy
    m.Add(z <= 0.70*LL + 0.60*LM + 0.65*LR + 0.93*RL + 0.72*RM + 0.70*RR, name="([L,M],[R,R])");

    # z <= Chance of scoring if keeper follows ([L,R],[R,R]) strategy
    m.Add(z <= 0.96*LL + 0.72*LM + 0.61*LR + 0.93*RL + 0.72*RM + 0.70*RR, name="([L,R],[R,R])");
    
    # Chances of kicker actions (Footedness, Aim direction) pairs add up to 1.
    m.Add(LL + LM + LR + RL + RM + RR == 1);
    
    return m, m.variables()

In [ ]:
m,x = penalty_kick_game()
solve(m)

<div>
  <p>Furthermore, instead of defining another LP, the keeper's optimal mixed strategy can be derived from the LP's dual values.</p>
</div>

In [ ]:
for c in m.constraints():
    print(c.name(),":", c.dual_value())

<div>
  <p><b>Q33:</b> Given the information from the dual and the solution, why can't the keeper use the values obtained from the dual for defending against a left footed kicker?</p>

  <p><b>A:</b><font color='blue'> The LP never uses a left-footed kicker, and as such has not completely solved the left-footed kicker sub problem. As such, the requirements for the strong duality theorem are not satisfied for the left-footed kicker subgame, and the dual values are not guaranteed to be optimal.</font></p>
</div>

<div>
  <font color='red'><p>Can this even be called a full set of instructions? The Left footed kicker instructions are incorrect and therefore omitted.</p></font>

  <p><b>Q34:</b> Based on the optimal solution and dual values to the full LP model, compile a <b>complete set of instructions</b> for the players in equilibrium.</p>

  <p><b>A:</b></p>
  <font color='blue'><ul>
    <li>The kicker is always <b>right-footed</b> and kicks to the <b>left</b> and <b>right</b> with probabilities: $p_L = 0.0968$ and $p_R = 0.9032$</li>
    <li>If the kicker is <b>right-footed</b>, the keeper <b>dives to the middle or right</b> with probabilities: $q_M = 0.7419$ (obtained by adding the $[R,M]$ keeper constraint dual values) and $q_R = 0.2581$</li>
  </ul></font>
</div>

<div>
  <p><b>Q35:</b> Do these instructions match those you described in <b>Q29</b>?</p>

  <p><b>A:</b><font color='blue'> Yes or no. If they argued yes, they should have stated that the players would act in the same way if they were given either set of instructions. If they argued no, they should have stated that the keeper's instructions from the full LP do not account for left-footed kickers.</font></p>
</div>

<a id="Part 4: Weaknesses of Mixed Strategies"></a>
<div>
  <br>
  <h2>Part 4: Weaknesses of Mixed Strategies</h2>

  <p>Unlike in the demos so far, opponent players will not always be constrained to using mixed strategies. If a keeper were to observe that all right-footed kickers were kicking to the right, they would likely adjust their strategy to prevent right footed kickers from scoring on the right. In such a way, the keeper could adopt a strategy that lowers the efficacy of your chosen mixed strategy.</p>

  <p>One such way of countering mixed strategies is <i>fictitious play</i>. Fictitious play is a learning rule where it is assumed that the opponent is using a stationary strategy. Fictitious play is run as follows:</p>

  <ul>
    <li>Keep track of the empirical frequency of the opponents played strategies.</li>
    <li>Make a decision that best counters your opponent's strategy so far.</li>
  </ul>

  <p>As the tracked empirical frequency better represents the opponents strategy, the player more effectively plays against their opponent.</p>
</div>

<div>
  <p><b>Q36:</b> In the penalty kick game, what would a keeper using fictitious play keep track of?</p>

  <p><b>A:</b><font color='blue'> The keeper would <b>separately track</b> the frequency of the kicker's kick directions (left, middle, right) for <b>left and right-footed kickers</b>.</font></p>
</div>

<div style="align: left; text-align:center;">
  <table style="width:60%">
    <tr>
      <th>Scoring Probability</th>
      <th>LL</th>
      <th>LM</th>
      <th>LR</th>
      <th>ML</th>
      <th>MM</th>
      <th>MR</th>
      <th>RL</th>
      <th>RM</th>
      <th>RR</th>
    </tr>
    <tr>
      <td><b>Right-footed kicker</b></td>
      <td>$0.55$</td>
      <td>$0.65$</td>
      <td>$0.93$</td>
      <td>$0.74$</td>
      <td>$0.60$</td>
      <td>$0.72$</td>
      <td>$0.95$</td>
      <td>$0.73$</td>
      <td>$0.70$</td>
    </tr>
    <tr>
      <td><b>Left-footed kicker</b></td>
      <td>$0.67$</td>
      <td>$0.70$</td>
      <td>$0.96$</td>
      <td>$0.74$</td>
      <td>$0.60$</td>
      <td>$0.72$</td>
      <td>$0.87$</td>
      <td>$0.65$</td>
      <td>$0.61$</td>
    </tr>
  </table>
  <div class="caption">Table $1$: The Column headers indicate kicker kick, keeper dive directions. </div>
</div> 

<br>

<div>
  <p><b>Q37:</b> Suppose the kicker is left-footed, and that the empirical frequencies of previously played left-footed kickers' choices suggest left, middle, right kicking chances of $p_L = 0.17$, $p_M = 0.17$, $p_R = 0.66$. What would the keeper's action be, if they were using fictitious play?</p>

  <p><b>A:</b><font color='blue'> $r_i$ denotes the predicted risk of being scored on for diving in direction $i$):<br>
$r_L : 0.67*0.17 + 0.74*0.17 + 0.87*0.66 = 0.8139$<br>
$r_M : 0.70*0.17 + 0.60*0.17 + 0.65*0.66 = 0.65$<br>
$r_R : 0.96*0.17 + 0.72*0.17 + 0.61*0.66 = 0.6882$</font></p>

  <font color='blue'><p>The keeper would stay in the middle, as $r_M = \min(r_L, r_M, r_R)$</p></font>
</div>

<div>
  <p><b>Q38: Use the same mixed strategy for each of the $3$ different rounds of the penalty kick game below.</b> The rounds have $50$, $100$, and $200$ iterations.</p>

  <p>Figure $3$ is now also accessible for you to view. Figure $3$ shows the keeper's perceived risk of taking each position.</p>

  <p>Analyze your results. Is there anything interesting about Figure $3$'s values? If so, did running more iterations increase that behaviour? Is there a correlation between the keeper's actions (viewable in Figure $4$), and how the values in Figure $3$ compare?</p>

  <p>If you are having trouble, there is an extra cell below that will run the game with even more iterations to give you more data.</p>

  <p><b>A:</b><font color='blue'> Answers will vary. The keeper is not guaranteed to do better, but seeing how the perceived risks converge to certain values (Towards the true likelihoods of being scored on for each position) over time in Figure $3$ is pretty cool. Students should observe that the convergence on values in figure $3$ is easier to see with more iterations. Figure $4$ may show how the keeper adjusted their strategy according to the kicker results.</font></p>
</div>

In [ ]:
# Shows the game so that you can play it
show(demo4)
show(demo5)
show(demo6)

In [ ]:
# Optional cell for running demo with more iterations.
show(demo7)

<div>
  <p><b>Q39:</b> What would the play of a keeper using fictitious play look like if their perceived risks of being scored on have stabilized?

  <p><b>A:</b><font color='blue'> If the perceived risks were stable and unequal, then the keeper would likely appear to be using a pure strategy. If the perceived risks were equal, then they could also appear to be using a mixed strategy split between the positions with equal perceived risks.</font></p>
</div>

<div>
  <p><b>Q40: Now try using a diifferent mixed strategy for each of the $3$ rounds of the game below.</b> This time, the keeper will behave in a different manner. Did you notice any correlation between the keeper's actions and the mixed strategy you used?</p>

  <p><b>A:</b><font color='blue'> The keeper should appear to use a pure strategy like mentioned in <b>Q39</b>. Students may notice that the strategy seems to counter their mixed strategy.</font></p>
</div>

In [ ]:
# Shows the game so that you can play it
show(demo8)
show(demo8)
show(demo8)

<div>
  <p><b>Q41:</b> The demos above actually had the keeper cheat! The keeper reveiceved the chances used for your mixed strategy and made a pure strategy to counter it. Why might this be similar to the result of fictitious play against a mixed strategy with infinite previous iterations?</p>

  <p><b>A:</b><font color='blue'> As you increase the amount of iterations played, you would expect the empirical frequency of the played strategies to approach the actual chances of each pure strategy being played. As such, with the frequencies in a state where they are almost constant, the problem of minimizing the risk (from the keeper's perspective) will continuously yield the same result (therefore allowing the choice made to be constant; a pure strategy).</font></p>
</div>

<div>
  <p><b>Q42:</b> Suppose you are again, no longer bound to using a mixed strategy. Is there a way you could exploit the keeper's fictitious play?</p>

  <p><b>A:</b><font color='blue'> By keeping track of the keeper's perceived risks, you could consistently play the actions that would maximize your chances of scoring against the keeper's next move.</font></p>
</div>

### Alternative Table 1: 

| Scoring Probability |  LL  |  LM  |  LR  |  ML  |  MM  |  MR  |  RL  |  RM  |  RR  |
|:-------------------:|:----:|:----:|:----:|:----:|:----:|:----:|:----:|:----:|:----:|
| Right-footed Kicker | 0.55 | 0.65 | 0.93 | 0.74 | 0.60 | 0.72 | 0.95 | 0.73 | 0.70 |
| Left-footed Kicker  | 0.67 | 0.70 | 0.96 | 0.74 | 0.60 | 0.72 | 0.87 | 0.65 | 0.61 |

<center>Table 1: The column headers indicate kicker kick, keeper dive directions.</center>

### Left-footed Kicker Subgame Alternative Matrix:
| Left-Footed Kicker  | Keeper Dives Left | Keeper Dives Middle | Keeper Dives Right |
|:-------------------:|:-----------------:|:-------------------:|:------------------:|
|  Kicker Kicks Left  |   +0.67 / -0.67   |    +0.70 / -0.70    |   +0.96 / -0.96    |
| Kicker Kicks Middle |   +0.74 / -0.74   |    +0.60 / -0.60    |   +0.72 / -0.72    |
| Kicker Kicks Right  |   +0.87 / -0.87   |    +0.65 / -0.65    |   +0.61 / -0.61    |

### Right-footed Kicker Subgame Alternative Matrix:
| Right-Footed Kicker | Keeper Dives Left | Keeper Dives Middle | Keeper Dives Right |
|:-------------------:|:-----------------:|:-------------------:|:------------------:|
|  Kicker Kicks Left  |   +0.55 / -0.55   |    +0.65 / -0.65    |   +0.93 / -0.93    |
| Kicker Kicks Middle |   +0.74 / -0.74   |    +0.60 / -0.60    |   +0.72 / -0.72    |
| Kicker Kicks Right  |   +0.95 / -0.95   |    +0.73 / -0.73    |   +0.70 / -0.70    |

### Right-footed Kicker Subgame Alternative Matrix w/ Middle Dropped:
| Right-Footed Kicker | Keeper Dives Left | Keeper Dives Middle | Keeper Dives Right |
|:-------------------:|:-----------------:|:-------------------:|:------------------:|
|  Kicker Kicks Left  |   +0.55 / -0.55   |    +0.65 / -0.65    |   +0.93 / -0.93    |
| Kicker Kicks Right  |   +0.95 / -0.95   |    +0.73 / -0.73    |   +0.70 / -0.70    |

### Left-footed Kicker Subgame Alternative Matrix w/ Middle Dropped:
| Left-Footed Kicker  | Keeper Dives Left | Keeper Dives Middle | Keeper Dives Right |
|:-------------------:|:-----------------:|:-------------------:|:------------------:|
|  Kicker Kicks Left  |   +0.67 / -0.67   |    +0.70 / -0.70    |   +0.96 / -0.96    |
| Kicker Kicks Right  |   +0.87 / -0.87   |    +0.65 / -0.65    |   +0.61 / -0.61    |

### Right-footed Kicker Subgame Alternative Matrix w/ Middle and Dive Left Dropped:
| Right-Footed Kicker | Keeper Dives Middle | Keeper Dives Right |
|:-------------------:|:-------------------:|:------------------:|
|  Kicker Kicks Left  |    +0.65 / -0.65    |   +0.93 / -0.93    |
| Kicker Kicks Right  |    +0.73 / -0.73    |   +0.70 / -0.70    |